In [1]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
import numpy as np

# Making a dataset

In [2]:
df = pd.read_csv("/kaggle/input/false-statements/false_statements.csv")
del df['answer']
ds = Dataset.from_pandas(df)
ds = ds.train_test_split(test_size=0.15)
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'statement'],
        num_rows: 18703
    })
    test: Dataset({
        features: ['text', 'statement'],
        num_rows: 3301
    })
})

# Initializing model and tokenizer

In [3]:
model_checkpoint = "t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

# Setting the training args

In [4]:
def tokenize_function(examples):
    # Ensure text and statement are lists of strings
    texts = [str(text) for text in examples["text"]]
    statements = [str(statement) for statement in examples["statement"]]
    
    # Tokenize the inputs
    model_inputs = tokenizer(texts, max_length=256, truncation=True, padding="max_length")
    
    # Tokenize the targets
    labels = tokenizer(statements, max_length=128, truncation=True, padding="max_length")
    
    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs

# Apply the tokenization to your dataset
tokenized_ds = ds.map(tokenize_function, batched=True, remove_columns=["text", "statement"])

batch_size = 40
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-FT-false_statements",
    evaluation_strategy = "epoch",
    learning_rate=2e-3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=50,
    predict_with_generate=True,
    push_to_hub=False,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Map:   0%|          | 0/18703 [00:00<?, ? examples/s]

Map:   0%|          | 0/3301 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [1]:
!pip install wandb
import wandb
wandb.login(key='ENTER_YOUR_KEY')

In [6]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['test'],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: newzns1710 (edu_oro). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240909_204613-unwatccc
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run t5-small-FT-false_statements
wandb: ⭐️ View project at https://wandb.ai/edu_oro/huggingface
wandb: 🚀 View run at https://wandb.ai/edu_oro/huggingface/runs/unwatccc
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.

Epoch,Training Loss,Validation Loss
1,No log,0.194476
2,No log,0.191505
3,0.269900,0.195078
4,0.269900,0.199678
5,0.152900,0.204919
6,0.152900,0.208098
7,0.120200,0.218158
8,0.120200,0.224009
9,0.097400,0.234437
10,0.097400,0.242877


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=11700, training_loss=0.04670302015593928, metrics={'train_runtime': 14621.1576, 'train_samples_per_second': 63.959, 'train_steps_per_second': 0.8, 'total_flos': 6.32824428232704e+16, 'train_loss': 0.04670302015593928, 'epoch': 50.0})